# Day 05: Hydrothermal Venture

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```text
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```text
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. **At how many points do at least two lines overlap?**

In [1]:
# Python imports
from pathlib import Path
from typing import Iterable, List, Tuple

import numpy as np

# Paths to data
testpath = Path("day05_test.txt")
datapath = Path("day05_data.txt")

It makes sense to load x, y co-ordinates as tuples, and the start, end points as a collection of some form.

In [2]:
def load_input(fpath: Path) -> List[List[Tuple]]:
    """Return list of line segment co-ordinates
    
    :param fpath:  Path to data file
    """
    segments = []  # holds start, end co-ordinate pairs
    
    with fpath.open("r") as ifh:
        for line in [_.strip().split(" -> ") for _ in ifh.readlines() if len(_.strip())]:
            start = tuple([int(_) for _ in line[0].split(",")])  # start co-ordinate
            end = tuple([int(_) for _ in line[1].split(",")])  # end co-ordinate
            segments.append([start, end])
            
    return segments

The seafloor looks like a 2D array, and the overlap counts are sensible array element values, so we'll use an `np.array` to represent this, for convenience and to take advantage of whatever methods we can.

We'll wrap this in a class, so we can add methods/attributes to calculate the puzzle answer (`.overlap`) and check we've got the correct seafloor map (`__str__()`).

In [3]:
class SeaFloor:
    
    """Representation of a seafloor with hydrothermal vents"""
    
    def __init__(self, segments: List[List[Tuple]]) -> None:
        """Set up seabed with vents defined by segments
        
        :param segments: list of start, end co-ordinates for vents
        """
        # Determine floor size from vent list, and create clear seafloor 
        xlist, ylist = [], []
        for start, end in segments:
            xlist.extend([start[0], end[0]])
            ylist.extend([start[1], end[1]])
        self.__setup_floor((max(ylist) + 1, max(xlist) + 1))
        
        # Add vents to the seafloor
        self.add_vents(segments)
        
    def __setup_floor(self, shape: Tuple[int, int]) -> None:
        """Create empty array to represent seafloor
        
        :param shape:  array size (y, x)
        """
        self._ventcounts = np.zeros(shape)
        
    def add_vents(self, segments: [List[List[Tuple]]]) -> None:
        """Add hydrothermal vents from a set of line segments
        
        :param segments:  start and end co-ordinates for vents
        """
        for start, end in segments:
            self.add_vent(start, end)
        
    def add_vent(self, start: Tuple[int, int], end: Tuple[int, int]) -> None:
        """Add hydrothermal vents from a pair of co-ordinates
        
        :param start:  start x, y coordinate
        :param end:  end x, y coordinate
        """
        # Split start, end coords for later use
        startx, starty = start
        endx, endy = end

        # Horizontal and vertical lines only
        if startx == endx or starty == endy:
            # Order start/end for ease of calculation
            startx, endx = min(startx, endx), max(startx, endx)
            starty, endy = min(starty, endy), max(starty, endy)

            if start == end:  # single point
                self._ventcounts[starty, startx] += 1
            elif startx != endx:  # vertical line
                for xpos in range(startx, endx+1):
                    self._ventcounts[starty, xpos] += 1
            elif starty != endy:  # horizontal line
                for ypos in range(starty, endy+1):
                    self._ventcounts[ypos, startx] += 1
        
    def __str__(self) -> str:
        """Return seafloor as string representation of vent counts"""
        outstr = []
        for row in self._ventcounts:
            outstr.append(" ".join(["." if _ == 0 else str(int(_)) for _ in row]))
        return "\n".join(outstr) + "\n"
    
    @property
    def overlaps(self):
        """Return number of points on seafloor with at least two vents"""
        return (self._ventcounts > 1).sum()

Then, solving the test puzzle

In [4]:
segments = load_input(testpath)
seafloor = SeaFloor(segments)
print(seafloor)
seafloor.overlaps

. . . . . . . 1 . .
. . 1 . . . . 1 . .
. . 1 . . . . 1 . .
. . . . . . . 1 . .
. 1 1 2 1 1 1 2 1 1
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
2 2 2 1 1 1 . . . .



5

And, for the real puzzle data:

In [5]:
segments = load_input(datapath)
seafloor = SeaFloor(segments)
seafloor.overlaps

6564

## Puzzle 2

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:

```text
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

Having used a class to represent the seafloor, the additional code to process diagonal lines is pretty minor. We add it to the `add_vent()` method.

In [6]:
class SeaFloor:
    
    """Representation of a seafloor with hydrothermal vents"""
    
    def __init__(self, segments: List[List[Tuple]]) -> None:
        """Set up seabed with vents defined by segments
        
        :param segments: list of start, end co-ordinates for vents
        """
        # Determine floor size from vent list, and create clear seafloor 
        xlist, ylist = [], []
        for start, end in segments:
            xlist.extend([start[0], end[0]])
            ylist.extend([start[1], end[1]])
        self.__setup_floor((max(ylist) + 1, max(xlist) + 1))
        
        # Add vents to the seafloor
        self.add_vents(segments)
        
    def __setup_floor(self, shape: Tuple[int, int]) -> None:
        """Create empty array to represent seafloor
        
        :param shape:  array size (y, x)
        """
        self._ventcounts = np.zeros(shape)
        
    def add_vents(self, segments: [List[List[Tuple]]]) -> None:
        """Add hydrothermal vents from a set of line segments
        
        :param segments:  start and end co-ordinates for vents
        """
        for start, end in segments:
            self.add_vent(start, end)
        
    def add_vent(self, start: Tuple[int, int], end: Tuple[int, int]) -> None:
        """Add hydrothermal vents from a pair of co-ordinates
        
        :param start:  start x, y coordinate
        :param end:  end x, y coordinate
        """
        # Split start, end coords for later use
        startx, starty = start
        endx, endy = end

        # Horizontal and vertical lines only
        if startx == endx or starty == endy:
            # Order start/end for ease of calculation
            startx, endx = min(startx, endx), max(startx, endx)
            starty, endy = min(starty, endy), max(starty, endy)

            if start == end:  # single point
                self._ventcounts[starty, startx] += 1
            elif startx != endx:  # vertical line
                for xpos in range(startx, endx+1):
                    self._ventcounts[starty, xpos] += 1
            elif starty != endy:  # horizontal line
                for ypos in range(starty, endy+1):
                    self._ventcounts[ypos, startx] += 1
        else:  # diagonal lines
            if endx > startx:
                xrange = range(startx, endx + 1)
            else:
                xrange = range(startx, endx - 1, -1)
            if endy > starty:
                yrange = range(starty, endy + 1)
            else:
                yrange = range(starty, endy - 1, -1)
            for xpos, ypos in zip(xrange, yrange):
                self._ventcounts[ypos, xpos] += 1
                    
        
    def __str__(self) -> str:
        """Return seafloor as string representation of vent counts"""
        outstr = []
        for row in self._ventcounts:
            outstr.append(" ".join(["." if _ == 0 else str(int(_)) for _ in row]))
        return "\n".join(outstr) + "\n"
    
    @property
    def overlaps(self):
        """Return number of points on seafloor with at least two vents"""
        return (self._ventcounts > 1).sum()

Checking with the test data:

In [7]:
segments = load_input(testpath)
seafloor = SeaFloor(segments)
print(seafloor)
seafloor.overlaps

1 . 1 . . . . 1 1 .
. 1 1 1 . . . 2 . .
. . 2 . 1 . 1 1 1 .
. . . 1 . 2 . 2 . .
. 1 1 2 3 1 3 2 1 1
. . . 1 . 2 . . . .
. . 1 . . . 1 . . .
. 1 . . . . . 1 . .
1 . . . . . . . 1 .
2 2 2 1 1 1 . . . .



12

And with the real puzzle data:

In [8]:
segments = load_input(datapath)
seafloor = SeaFloor(segments)
seafloor.overlaps

19172